In [6]:
import pandas as pd 
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdChemReactions
from rdkit.Chem import AllChem
import os
import PySimpleGUI as sg
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use('TkAgg')

df = pd.read_csv("CSV/materials.csv",index_col = "Tag")
materials = df.index.tolist()
properties = df.columns.tolist()[3:] #官能基の情報のみ


#機械学習用データフレーム 
try:
    products_df = pd.read_csv("CSV/products.csv")
except:
    products_df = pd.DataFrame(columns=["Name","Object Val"]+properties)

# GUIのレイアウト
sg.theme("DarkBrown")

layout = [[sg.Text('PRODUCTS', font=('Constantia',20))],
          [sg.Text("")],
          [sg.Button("Calculate"),sg.Button("Delete"),sg.Button("Elastic Net")],
          [sg.Text("Sample Name",size=(15, 1)), sg.Input(size=(50, 1),key = "sample_name" )],
          [sg.Text("Value",size=(15, 1)), sg.Input(size=(50, 1),key = "val" )],
          [sg.Text('_'  * 70)], #横線区切り
          [sg.Column(layout=[
                  [sg.Combo(materials, size=(20, 1), key=f'material_{n}'),
                   sg.Input("0", size=(20, 1), key=f"feed_{n}")] for n in range(15)],
              size=(400, 400)  # 列全体のサイズ
              ),
           sg.Table(headings =["Name"],values = [[elem] for elem in products_df["Name"].tolist()],
                    key ="output_table",
                    size=(10,25))
          ]]
# ウインドウの出現位置を指定
win_location = (0, 0)

# モニターの解像度を取得
screen_width, screen_height = sg.Window.get_screen_size()

# ウィンドウのサイズをモニターの半分に設定
window_size = (screen_width // 2, screen_height)

window = sg.Window("PRODUCTS", 
                   layout, 
                   size=window_size,
                   resizable=True,
                   location=win_location)



while True:
    
    event, values = window.read()
    
    if event == sg.WINDOW_CLOSED:
        break
        
    if event == "Calculate":

        #出力用dict
        output={} 
        for property in properties:
            calc_val = []
            feed_sum = []
            
            for i in range(15):
                material_name = values[f"material_{i}"]
                if material_name != "":  #materialnameに入力されている場合のみデータ参照可能
                    feed = float(values[f"feed_{i}"])
                    calc_val.append(df.loc[material_name,property]*feed)
                    feed_sum.append(feed)

            #目的変数値の記録
            output["Name"] = [values["sample_name"]]
            #目的変数値の記録
            output["Object Val"] = [values["val"]]
            #官能基情報の記録
            output[property] = [sum(calc_val)/sum(feed_sum)]

        functional_group = [elem.replace("(mmol/g)","") for elem in output.keys()][2:]
        quantity = [elem[0] for elem in output.values()][2:]
        
        plt.close()
        fig = plt.figure(figsize=(8, 6)) 
        plt.get_current_fig_manager().window.wm_geometry("+550+0")
        plt.barh(functional_group,quantity,color='gray')
        plt.show(block=False)
        plt.title(output["Name"][0])


        
        #データフレームに新しいデータ行を追加
        new_row = pd.DataFrame(output)
        products_df = pd.concat([products_df,new_row])
        window["output_table"].update([[elem] for elem in products_df["Name"].tolist()])

    
    if event == "Delete":
        try:
            products_df = products_df.drop(products_df.index[-1])
            window["output_table"].update([[elem] for elem in products_df["Name"].tolist()])
        except:
            pass


    if event == "Elastic Net":
        
        y = products_df["Object Val"]
        X = products_df.drop(columns=["Object Val","Name","logP"])
        properties = [word.replace(" (mmol/g)","") for word in X.columns.to_list()]
        
        y_ = y.to_numpy().astype(float)
        X_ = X.to_numpy().astype(float)
        
        from modules import bayes_EN as ben
        ben.Elastic_Net(properties,X_,y_,0.8,100)

products_df.to_csv("CSV/products.csv",index=False)
window.close()

In [7]:
products_df

,Name,Object Val,logP,carboxyl (mmol/g),carboxylate (mmol/g),hydroxyl (mmol/g),ether (mmol/g),ester (mmol/g),epoxy (mmol/g),amide (mmol/g),...,amine (mmol/g),ammonium (mmol/g),phenyl (mmol/g),methyl (mmol/g),methylene (mmol/g),methyne (mmol/g),quat_carbon (mmol/g),vinyl (mmol/g),asetylene (mmol/g),siloxane (mmol/g)
0,b.p,7.48,7.48660,0.000000,0.000000,0.000000,0.0,2.953547,0.000000,0.0,...,0.000000,0.000000,0.000000,5.907093,53.163839,0.000000,2.953547,0.000000,0.0,0.0
1,b.p,3.39,3.43990,0.000000,0.000000,0.000000,0.0,4.497859,0.000000,0.0,...,0.000000,0.000000,0.000000,17.991436,17.991436,8.995718,13.493577,0.000000,0.0,0.0
2,b.p,3.44,3.44150,0.000000,0.000000,0.000000,0.0,5.042712,0.000000,0.0,...,0.000000,0.000000,0.000000,15.128135,30.256271,5.042712,5.042712,0.000000,0.0,0.0
3,b.p,0.37,0.37640,13.876747,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,13.876747,13.876747,0.000000,0.000000,0.0,0.0
4,b.p,0.60,0.60590,0.000000,0.000000,6.936256,0.0,6.936256,0.000000,0.0,...,0.000000,0.000000,0.000000,13.872512,13.872512,6.936256,6.936256,0.000000,0.0,0.0
5,b.p,0.62,0.62390,0.000000,0.000000,0.000000,0.0,7.034624,7.034624,0.0,...,0.000000,0.000000,0.000000,7.034624,21.103873,7.034624,7.034624,0.000000,0.0,0.0
6,b.p,0.78,0.78670,0.000000,0.000000,0.000000,0.0,6.360797,0.000000,0.0,...,6.360797,0.000000,0.000000,19.082391,19.082391,0.000000,6.360797,0.000000,0.0,0.0
7,b.p,-0.63,-0.63040,0.000000,0.000000,0.000000,0.0,6.320274,0.000000,0.0,...,0.000000,6.320274,0.000000,18.960821,18.960821,0.000000,6.320274,0.000000,0.0,0.0
8,b.p,-0.95,-0.95830,0.000000,14.073605,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,14.073605,14.073605,0.000000,0.000000,0.0,0.0
9,b.p,6.33,6.33250,3.515136,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,3.515136,56.242179,0.000000,0.000000,0.000000,0.0,0.0


In [14]:
# サブプロットの配置を設定
num_rows = 2
num_cols = 5

# ループで10個のサブプロットを生成して表示
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 6))
axes.flat


In [26]:

import matplotlib.pyplot as plt

# サブプロットの配置を設定
num_rows = 4
num_cols = 5

# ループで10個のサブプロットを生成して表示
fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 8))

for ax, elem in zip(axes.flat,products_df.columns[2:]):
    x = products_df[elem].tolist()
    y = products_df["Object Val"].tolist()
    ax.scatter(x, y, s=3, marker='o', color='blue')  # 仮のデータをプロット
    ax.set_title(elem,fontsize=10)
    ax.set_xlabel('mmol/g',fontsize=10)
    ax.set_ylabel('Object Value',fontsize=10)
# サブプロット間のスペースを調整
plt.tight_layout()

# グラフを表示
plt.show()
    
    
